# Dynamics Topics

Notebook based on code retrieved from [MaartenGr/BERTopic](https://github.com/MaartenGr/BERTopic)

Remember to enable GPU by `Runtime>Change runtime type>Hardware accelerator (GPU)`

In [1]:
!pip install bertopic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 87.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 4.9 MB/s eta 0:00:00


First, we need to load in the data and do some very basic cleaning. For example, I am not interested in his re-tweets for this use-case:

In [2]:
import re
import pandas as pd
trump = pd.read_csv('../dataprocessed/datos_preprocesados_2.csv')
trump.review = trump.apply(lambda row: re.sub(r"http\S+", "", row.review).lower(), 1)
trump.review = trump.apply(lambda row: " ".join(filter(lambda x:x[0]!="@", row.review.split())), 1)
trump.review = trump.apply(lambda row: " ".join(re.sub("[^a-zA-Z]+", " ", row.review).split()), 1)
trump['timestamp_formatted'] = pd.to_datetime(trump['timestamp_formatted'], dayfirst=True)
trump['timestamp_formatted'] = trump['timestamp_formatted'].dt.strftime('%m/%d/%Y')
timestamps = trump.timestamp_formatted.to_list()
tweets = trump.tk_review.to_list()
print(len(tweets),"tweets are ready!")

590 tweets are ready!


In [3]:
trump.head()

,model_name,Price (Dollar),Storage (GB),review,timestamp,stars,country,language,tk_review,timestamp_nanosec,normalized_timestamp_nanosec,timestamp_formatted
0,iPhone XS,182.00,64,i received the phone within days of ordering t...,2022-02-05,1,the United States,en,"['received', 'phone', 'within', 'day', 'orderi...",1644019200000000000,0,01/01/1970
1,iPhone XS Max,234.95,64,months later update still acts like i bought i...,2019-12-20,5,the United States,en,"['month', 'later', 'update', 'still', 'act', '...",1576800000000000000,0,01/01/1970
2,iPhone 15 Plus,1098.95,128,phone is great but the price is above msrp wit...,2023-11-27,1,the United States,en,"['phone', 'great', 'price', '200', 'msrp', 're...",1701043200000000000,0,01/01/1970
3,iPhone 11,599.00,64,fast shipping very nice phone read more,2020-07-28,5,the United States,en,"['fast', 'shipping', 'nice', 'phone', 'read']",1595894400000000000,0,01/01/1970
4,iPhone 12 Pro,840.13,128,amazing defenitly worth it read more,2020-11-05,5,the United States,en,"['amazing', 'defenitly', 'worth', 'read']",1604534400000000000,0,01/01/1970


Some tweets...

In [4]:
for tweet in tweets[:10]:
  print("> Tweet:",tweet)

> Tweet: ['received', 'phone', 'within', 'day', 'ordering', 'packaging', 'fine', 'initially', 'appeared', 'great', 'condition', 'however', 'checked', 'battery', 'health', 'first', 'day', 'stated', 'important', 'battery', 'message', 'battery', 'health', 'significantly', 'degraded', 'maximum', 'capacity', 'battery', 'problem', 'thought', 'one', 'reason', 'bought', 'amazon', 'phone', 'advertised', 'battery', 'exceeds', 'capacity', 'relative', 'new', 'guaranteed', 'amazon', 'eligible', 'replacement', 'refund', 'within', 'day', 'receipt', 'satisfied', 'well', 'clearly', 'satisfied', 'receive', 'phone', 'significantly', 'degraded', 'battery', 'called', 'amazon', 'customer', 'service', 'explain', 'problem', 'request', 'replacement', 'day', 'received', 'phone', 'representative', 'helpful', 'told', 'would', 'send', 'replacement', 'immediately', 'even', 'upgrade', 'shipment', 'method', 'extra', 'charge', 'also', 'told', 'conversation', 'writing', 'use', 'box', 'replacement', 'come', 'return', 'o

Then, we need to extract the global topic representations by simply creating and training a BERTopic model:

In [5]:
from bertopic import BERTopic
topic_model = BERTopic(min_topic_size=5, verbose=True)
topics, _ = topic_model.fit_transform(tweets)

2024-12-19 22:04:26,757 - BERTopic - Embedding - Transforming documents to embeddings.


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/19 [00:00<?, ?it/s]

2024-12-19 22:04:36,218 - BERTopic - Embedding - Completed ✓
2024-12-19 22:04:36,223 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-12-19 22:04:46,567 - BERTopic - Dimensionality - Completed ✓
2024-12-19 22:04:46,569 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-12-19 22:04:46,605 - BERTopic - Cluster - Completed ✓
2024-12-19 22:04:46,612 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-12-19 22:04:46,707 - BERTopic - Representation - Completed ✓


We can then extract most frequent topics:

In [6]:
topic_model.get_topic_info().head(10)

,Topic,Count,Name,Representation,Representative_Docs
0,-1,191,-1_phone_iphone_read_battery,"[phone, iphone, read, battery, great, new, con...","[['bought', 'refurbished', 'iphone', 'pro', 'm..."
1,0,90,0_phone_day_amazon_screen,"[phone, day, amazon, screen, battery, return, ...","[['received', 'phone', 'within', 'day', 'order..."
2,1,23,1_bezel_scratch_new_usba,"[bezel, scratch, new, usba, ctype, 233, read, ...","[['heavily', 'scratched', 'bezel', 'screen', '..."
3,2,19,2_refurbished_great_phone_used,"[refurbished, great, phone, used, read, expect...","[['quality', 'phone', 'pleasantly', 'surprised..."
4,3,18,3_new_skeptical_perfectly_brand,"[new, skeptical, perfectly, brand, hesitate, r...","[['somewhat', 'skeptical', 'pre-owned', 'item'..."
5,4,17,4_multiple_terrible_returned_deep,"[multiple, terrible, returned, deep, screen, s...","[['screen', 'multiple', 'deep', 'scratch', 're..."
6,5,15,5_work_looked_brand_well,"[work, looked, brand, well, like, new, read, b...","[['work', 'well', 'looked', 'like', 'brand', '..."
7,6,15,6_iphone_love_iphones_read,"[iphone, love, iphones, read, like, android, f...","[['iphones', 'iphone', 'iphone', 'iphone', 'ip..."
8,7,15,7_read_defenitly_good_hate,"[read, defenitly, good, hate, everyone, happy,...","[['good', 'read'], ['read'], ['read']]"
9,8,15,8_company_amazon_experience_renewed,"[company, amazon, experience, renewed, year, i...","[['bought', 'renewed', 'iphone', 'amazon', 'fr..."


## Topics over Time

From these topics, we are going to generate the topic representations at each timestamp for each topic.

We do this by simply calling topics_over_time and pass in his tweets, the corresponding timestamps, and the related topics.

There are a few important parameters that you should take note of, namely:

* `docs`
  * These are the tweets that we are using
* `topics`
  * The topics that we have created before
* `timestamps`
  * The timestamp of each tweet/document
* `global_tuning`
  * Whether to average the topic representation of a topic at time *t* with its global topic representation
* `evolution_tuning`
  * Whether to average the topic representation of a topic at time *t* with the topic representation of that topic at time *t-1*
* `nr_bins`
  * The number of bins to put our timestamps into. It is computationally inefficient to extract the topics at thousands of different timestamps. Therefore, it is advised to keep this value below 20.




In [7]:
topics_over_time = topic_model.topics_over_time(docs=tweets,
                                                topics=topics,
                                                timestamps=timestamps,
                                                global_tuning=True,
                                                evolution_tuning=True,
                                                nr_bins=20)

20it [00:00, 48.02it/s]


### Visualize Topics over Time

After having created our topics_over_time, we will have to visualize those topics as accessing them becomes a bit more difficult with the added temporal dimension.


To do so, we are going to visualize the distribution of topics over time based on their frequency. Doing so allows us to see how the topics have evolved over time. Make sure to hover over any point to see how the topic representation at time t differs from the global topic representation.

In [8]:
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=20)

In [23]:
from bertopic import BERTopic

top_n = 10

topics = topic_model.get_topics()
topic_words = []

for topic_id in topics:
    words = [word for word, _ in topics[topic_id][:top_n]]
    topic_words.append(words)

all_words = [word for words in topic_words for word in words]
unique_words = set(all_words)

topic_diversity = len(unique_words) / len(all_words)

print(f"Diversidad de Tópicos: {topic_diversity:.4f}")

Diversidad de Tópicos: 0.7500


In [27]:
import numpy as np

def calculate_perplexity(topics_dict):
    # Extraer probabilidades de cada palabra en cada tópico
    probabilities = []
    for topic_id in topics_dict:
        topic_probs = [prob for _, prob in topics_dict[topic_id]]
        probabilities.extend(topic_probs)

    # Convertir a array numpy
    probs = np.array(probabilities)

    # Normalizar probabilidades
    probs = probs / np.sum(probs)

    # Calcular perplejidad
    entropy = -np.sum(probs * np.log2(probs + 1e-10))
    perplexity = 2 ** entropy

    return perplexity

# Calcular perplejidad con los tópicos dados
perplexity = calculate_perplexity(topics)
print(f"Perplejidad del modelo: {perplexity:.4f}")

Perplejidad del modelo: 231.7842


In [43]:
from sklearn.metrics.pairwise import cosine_similarity
topic_embeddings = topic_model.topic_embeddings_
topic_similarity = cosine_similarity(topic_embeddings)
np.mean(np.mean(np.abs(topic_similarity), axis=0))

0.785188